# Assignment 3
## Part 1
### Theory Questions
    1) What are the diffrences between logistic regression and linear regression?
            
            Linear regression is for continous problems whereas logistic regression is a classifier.
        Logistic regression is a probabilistic approach. It calculates the probabilities of each class and tries to minimize
        its error. Linear regression is an analaytic approach. It tries to find a line that is fits best to the data points.
        This approaches results different objective functions. Logistic regression uses cross-entropy error function and
        linear regression uses mean squared error function.
        
    2)What are differences between logistic regression and naive bayes methods?
           
           Logistic regression tries to find posterior directly. This makes logistic regression a discriminitive model.
        Naive bayes first finds prior probability for each feature. Then, with the assumption of conditional independency
        of each variable, it finds posterior. This makes naive bayes a generative model.
            Naive bayes reaches to its asymptotic error faster. However, logistic regression ha a lower asymptotic error.
        This makes naive bayes a better algorithm for small datasets and makes logistic regression better for large datasets.
            As mentioned above, naive bayes makes the assumption of marginally conditionally independence for every feature
        Another diffrence is that logistic regression does not have such an assumption.

    3)Which of the following statements are true?
        
        True ones are:
                A two layer (one input layer, one output layer; no hidden layer) neural network
            can represent the XOR function.
            
                Any logical function over binary-valued (0 or 1) inputs x 1 andx 2 can be
            (approximately) represented using some neural network.
                
                The activation values of the hidden units in a neural network, with the sigmoid
            activation function applied at every layer, are always in the range (0, 1).
            It is hard to say anything on the last one. Activation values of the hidden units does not needs to be
            neccesarily between (0,1). But when application function is applied to activasions, outputs would range
            between(0,1). I added this one as true too because I didn't understand which one was asking.
            
    4)How to decide the number of hidden layers and nodes in a hidden layer?
        
        They say that, in most cases, 1 hidden layer is sufdicent enough. The number of neurons in one layer should be
    between the output class size and feauture size.
    

### Part 2: Classification of Flowers using Neural Networks
### Owerview of the Problem
        Neural networks can express many data distributions. In this homework, we are expected to construct neural network to classify five class
    of flowers. First, we will implement a single layer neural network. Then, we extend the neural network to a multilayer network.
### Data Set
        We will use this https://www.kaggle.com/alxmamaev/flowers-recognition/downloads/flowers.zip/1 dataset. In addition to that
    Necva Bölücü hoca shared a code that converts pictures to vectors with 768 features. I have used that vectors as inputs.

### Algorithm
        Neural networks consists of layers. Each layer has an activation function and some number of neurons. Each neuron has a weight vector.
    In addition to that, a neural network has an objective function. The goal of training is to minimize objective function.
    It is really import to know the overall structure of a neural network in order to understand the algorithm. 
    
        The training algorithm has two main parts. These parts are feed forward and back propagation.We use these parts consecutively in order
    to update the weights of each neuron. We first feed forward, and then back propagete to find derivatives.
    


<img src="network.png" width="1000" height="500">

#### Feed Forward
&emsp;For each layer, one forward is finding the input of the next layer, with respect to its weights and activation function.<br>
Lets suppose that we are at layer j and find the inputs of the next layer. $w_{ji}$ denotes the i'th neurons weight vector and $z_j$ is the inputs<br>
of the the j'th layer and $z_{ki}$ is the i'th input of the next layer. $h_j()$ is j'th layers activation. Then one forward means finding the $z_k$ vector.<br>
$z_kj = h_j(z_j^T.w_{ji})$
&emps;Finding the inputs f each layer is feed forward. At last layer, we find the inputs of the error function.


#### Back Propagation
&emsp;In order to understand back propagation lets consider a middle layer.<br>
Lets suppose that $z_j$ is the inputs of the j'th layer and $a_k$ is the inputs of the activations of the layer after. I adopt the<br>
same notation above for the other parts. Lets say $E$ denotes the error function.<br>
$\dfrac{\partial E}{\partial w_{ji}} = \dfrac{\partial E}{\partial a_j}\dfrac{\partial a_j}{\partial w_{ji}}$<br>
lets say  $\dfrac{\partial E}{\partial a_j} = \delta_j$ and $\dfrac{\partial E}{\partial a_k} = \delta_k$ then<br>
$\dfrac{\partial a_j}{\partial w_{ji}} = z_j\implies\dfrac{\partial E}{\partial w_{ji}} = \delta_jz_{ji}$<br>
$\delta_j = \underset{K}{\Sigma}\dfrac{\partial E}{\partial a_k}\dfrac{\partial a_k}{\partial a_j}$<br>
$\delta_j = h'(a_j)\underset{K}{\Sigma}\delta_kw_{kj}$<br>
We can find the $\delta_{last}$ with help of feed forward. Back propagation is finding each weights derivative with respect to error<br>
function with recursively fiding the $\delta$ before starting from $\delta_{last}$.


### Implentation
#### Data Structures
&emsp;I have used data structures to define activasion functions. Each activason function has its class with its activation function<br>
and derivative.


In [ ]:
class Sigmoid:
	def __init__(self):
		return
	
	def activate(self, x):
		return 1.0/(1 + np.exp(-x)) 		
	
	def derivative(self,x):
		return  x * (1 - x)

class Tanh:
	def __init__(self):
		return 
	
	def activate(self, x):
		val1 = np.exp(x)
		val2 = np.exp(-x)
		return (val1 - val2) / (val1 + val2)

	def derivative(self, x):
		return 1-x**2
		
class ReLu:
	def __init__(self):
		return 
	
	def activate(self, x):
		if(x > 0):
			return x
		else:
			return 0
			
	def derivative(self, x):
		if(x > 0):
			return 1
		else:
			return 0
			
class NoActivation:
	def __init__(self):
		return 
	
	def activate(self, x):
		return x
			
	def derivative(self, x):
		return 1


I also have a layer class. And a network class which consists of layers.

In [ ]:
class Layer:
		# weightNum is the number of weights each layer has
		# neuronNum is neuron number
		# func is layers activation function class
		# bias is the bias vector of this layer   
		# inpV is the input vector for that layer
		def __init__(self, weightNum, neuronNum, func):
			self.bias = np.random.randn(neuronNum) * 0.01
			self.func = func
			self.inpV = None
			self.weights = np.random.randn(neuronNum, weightNum) * 0.01
			return	
		
		# returns the input of the next layer
		def forward(self):
			ret = np.zeros(len(self.weights))
			for neuron in range(len(self.weights)):
				z = np.dot(self.weights[neuron], self.inpV) + self.bias[neuron]
				ret[neuron] = self.func.activate(z)
			return ret
		
		# inpV are the input vector of the layer after this layer.
		# Finds the derivative of the layer.
		# Makes activations the derivative vector where activasions[i] is the derivative of the
		# i'th neurons activasion function with respect to input of that activation function. 	
		def layerDerivative(self, inpV):
			ret = np.array(inpV)
			for i in range(len(inpV)):
				ret[i] = self.func.derivative(inpV[i])
			return ret
		
		# inpV are the input vector of the layer after this layer.	
		# weightBefore is weights before this level
		# lambdaBefore is the lambda for
		# returns delta vector for that class 
		def findDelta(self, inpV, weightBefore, deltaBefore):
			der = self.layerDerivative(inpV)
			delta = np.dot(weightBefore.transpose(),deltaBefore)
			delta = np.multiply(delta, der)
			return delta
			
			
class NeuralNetwork:
	# NeuronNumV[i] is the number of neurons in i'th layer
	# FunctionV[i] is the activasion function in i'th layer
	# FunctionV elements can be "tanh", "sigmoid", "relu"
	# Default is a neuron with no activasion function
	# ClassNum is the number of output classes
	# If neuronNumV and functionV is an empty listi then it will create a single layer network.
	def __init__(self, neuronNumV, functionV, classNum, featureNum):
		relu = ReLu()
		sigmoid = Sigmoid()
		tanh = Tanh()
		default = NoActivation()
		self.layers = []
		fdict = {"relu": relu, "sigmoid": sigmoid, "tanh": tanh}
		if(len(neuronNumV) != len( functionV)):
			print("Each layer function must be specified")
			print("Error in NeuralNetwork class while initializing object")
			quit()
		for i in range(len(functionV)):
			func = fdict.get(functionV[i])
			if(func):
				functionV[i] = func
			else:
				functionV[i] = default
		if(len(neuronNumV) > 0):
			self.layers.append(Layer(featureNum, neuronNumV[0],functionV[0]))
			for i in range(1, len(neuronNumV)):
		 		self.layers.append(Layer(neuronNumV[i-1], neuronNumV[i],functionV[i]))
			self.layers.append(Layer(neuronNumV[-1], classNum, default))	# Last layer. Input of the cost function.
		else:
			self.layers.append(Layer(featureNum, classNum, default))	# Single layer network
	# feauters is the feature is a feature vector. It must have the same size of the featureNum above
	# y is the class number
	# feedforwards data		
	# returns last layers outputs
	def feedForward(self, features, y):
		self.layers[0].inpV = features
		for i in range(1,len(self.layers)):
			self.layers[i].inpV = self.layers[i-1].forward()
		return self.layers[-1].forward()
		
	# Backpropagates
	# Returns deltas. This deltas will be used for weight updating
	def backProp(self, vector, y):
		global error
		wder= []
		bder = []
		for layer in self.layers:
			wder.append(np.zeros(layer.weights.shape)) ## each neuron has its own lamda value
			bder.append(np.zeros(len(layer.bias.shape)))
		#print(vector)
		vector = softmax(vector)
		#print(vector)
		#quit()
		error = crossEntropyError(vector, y) + error
		#print( crossEntropyError(vector, y))
		delta = softmaxDelta(vector, y)
		wder[-1] = np.dot(self.layers[-1].weights.transpose(), delta)
		bder[-1] = delta	
		length = len(self.layers) - 1
		for i in range(length):
			cur = length - i - 1
			layer = self.layers[cur]
			inpBefore = self.layers[cur + 1].inpV
			weightBefore = self.layers[cur + 1].weights
			delta = layer.findDelta(inpBefore, weightBefore, delta)
			wder[cur] =  np.dot(self.layers[cur].weights.transpose(), delta)
			bder[cur] = delta
		return (wder, bder)
		
	# miniBatch is a tuple list corrosponds of input feautures and correct classes
	# return error for that batch
	def miniBatchUpdate(self, minibatch): 
		global rate
		wder = []
		bder = []
		for layer in self.layers:
			wder.append(np.zeros(layer.weights.shape)) ## each neuron has its own lamda value
			bder.append(np.zeros(layer.bias.shape))
		for inp in minibatch:
			feature = inp[0]
			y = inp[1]
			lastAct= self.feedForward(feature, y)
		#	print(lastAct); print("*****")
			(wderPass,bderPass) = self.backProp(lastAct, y)
			wder = [dw + dwp for dw,dwp in zip(wder, wderPass)]
			bder = [bw + bwp for bw, bwp in zip(bder, bderPass)]			
		wder = [layer/len(wder) for layer in wder]
		bder = [layer/len(bder) for layer in bder] 
		for i in range(len(self.layers)):
			self.layers[i].weights = self.layers[i].weights - rate*wder[i]
			self.layers[i].bias = self.layers[i].bias - rate*bder[i]
			
	def train(self, inp, miniBatchSize, epoch):
		global error
		for e in range(epoch):
			error = 0
			rd.shuffle(inp)
			miniBatches = [inp[k:k+miniBatchSize] for k in range(0,len(inp), miniBatchSize)]
			for batch in miniBatches:
				self.miniBatchUpdate(batch)
			error = error / len(inp)
			print("epoch is " + str(e) + "\terror is " + str(error))
	

#### Problems

&emsp;I have implemented the algorithm with mini batch gradient descent. Sometimes derivatives may become infinity or nan. To solve this problem<br>
I multiply the initialization weights with a constant. This makes derivatives to become reasonable numbers.
&emsp;Sometimes, epoch errors starts to stabilize soon. In that case, I decrement the learning rate and try again.